In [1]:
%%capture
!wget https://www.dropbox.com/scl/fi/uznju0fkgnwz1yi4cmgol/review-Vermont.json.gz?rlkey=4l3mithogu5c08x5lxe6pbb6t&st=wcprbsjo&dl=0
!wget https://www.dropbox.com/scl/fi/16vy3q077mz01n4r2nml1/meta-Vermont.json.gz?rlkey=zu5joaocmaqdg0p17oj14z5lz&st=kih98lc6&dl=0
!mv review-Vermont.json.gz?rlkey=4l3mithogu5c08x5lxe6pbb6t review-Vermont.json.gz
!mv meta-Vermont.json.gz?rlkey=zu5joaocmaqdg0p17oj14z5lz meta-Vermont.json.gz

In [2]:
print("Importando librerias")
import random
from io import BytesIO
from requests import get
import concurrent.futures
import gzip
import json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
print("* Librerias importadas * ")

Importando librerias
* Librerias importadas * 


# Data Creation

In [3]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)


metadata = list(parse("meta-Vermont.json.gz"))
reviews = list(parse("review-Vermont.json.gz"))

In [37]:
data = {
    'user_id' : [],
    'gmap_id' : [],
    'rating' : [],
    'text' : [],
    'img_url' : []
}

for review in reviews:
    has_img = review['pics'] is not None
    has_text = review['text'] is not None
    if has_img and has_text:
        has_good_text = len(review['text']) >= 100
        if not has_good_text: continue
        # business pic
        pics = review['pics']
        first_pic = pics[0]
        url_first_pic = first_pic['url'][0] # select the first pic

        # business text
        text = review['text']

        # reviews's rating
        rating = review['rating']

        # user id
        user_id = review['user_id']

        # business id (restaurant id)
        gmap_id = review['gmap_id']

        data['user_id'].append(user_id)
        data['gmap_id'].append(gmap_id)
        data['rating'].append(rating)
        data['text'].append(text)
        data['img_url'].append(url_first_pic)

df = pd.DataFrame(data)

In [7]:
image_data = df[['gmap_id', 'img_url']]
os.makedirs('images', exist_ok=True)
indexes_to_drop = []

"""
fuente: https://medium.com/analytics-vidhya/how-to-download-images-faster-using-multithreading-in-python-a31110468770
"""

def download_image(row):
    gmap_id, img_url = row.gmap_id, row.img_url
    specific_data = df[(df.gmap_id == gmap_id) & (df.img_url == img_url)]
    index = specific_data.index[0]
    prefix = 'images'
    img_path = f'{prefix}/{gmap_id}.png'

    if os.path.isfile(img_path): # caso en que exista
        print(f"- [  !  ] La imagen {img_path} ya esta en la carpeta")
        return
    response = get(img_url)

    if response.status_code == 200:
        curr_images = len(os.listdir('images'))
        print(f"- [ DONE ] ({curr_images}/{image_data.shape[0]}) Descargando y guardando imagen {gmap_id}.png con exito")
        img_bytes = response.content
        with open(img_path, 'wb') as img_file:
            img_file.write(img_bytes)

        df.at[index, 'img_url'] = img_path
    else:
        print(f"- [ FAIL ] Imagen {img_url} no puede ser descargada")
        indexes_to_drop.append(index)

def create_folder():
    print("Creando carpeta 'images/'")
    os.makedirs('images', exist_ok=True)

    t1 = time.perf_counter()

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor: # Pueden alterar la cantidad de workers segun el rendimiento
        for row in image_data.itertuples():
            executor.submit(download_image, row)

    t2 = time.perf_counter()

    print(f'Finished in {t2-t1} seconds')


In [8]:
create_folder()

Se truncaron las últimas líneas 5000 del resultado de transmisión.
- [ DONE ] (3461/15453) Descargando y guardando imagen 0x4cca7a6ef5383ae9:0x20577a732408e53d.png con exito
- [ DONE ] (3454/15453) Descargando y guardando imagen 0x4cb4ce5c551f18a7:0xdaf732dde1a48a87.png con exito
- [ DONE ] (3461/15453) Descargando y guardando imagen 0x89e02c85f21b4013:0x647e4adb955e5a9d.png con exito
- [ DONE ] (3460/15453) Descargando y guardando imagen 0x89e02c85f21b4013:0x647e4adb955e5a9d.png con exito
- [ DONE ] (3461/15453) Descargando y guardando imagen 0x4cca7084d662d9ed:0xc36154fa62e3515a.png con exito
- [ DONE ] (3460/15453) Descargando y guardando imagen 0x4cca7af9c174e1cf:0x8d157507941ec178.png con exito
- [ DONE ] (3461/15453) Descargando y guardando imagen 0x89e02bfe72e07899:0x71a0289466d99094.png con exito
- [ DONE ] (3460/15453) Descargando y guardando imagen 0x4cca7a6ef5383ae9:0x20577a732408e53d.png con exito
- [ DONE ] (3460/15453) Descargando y guardando imagen 0x4cca7a6ef5383ae9:0x2

In [10]:
# ojo, probablemente no todos los request sean efectivos,
# por lo tanto, es probable que algunas imagenes no se descarguen
len(os.listdir('images'))

3999

In [11]:
%%capture
!zip -r images.zip images

In [55]:
def image_in_folder(row):
    img_url = row.gmap_id + '.png'
    if img_url not in os.listdir('images'):
        return False
    return True

df['image_in_folder'] = df.apply(image_in_folder, axis=1)

In [59]:
df_filtered = df[df.image_in_folder]

In [61]:
df_filtered.drop(columns=['image_in_folder'], inplace=True)

<ipython-input-61-c04a6fb3ae26>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['image_in_folder'], inplace=True)


In [63]:
for row in df_filtered.itertuples():
    filename = row.gmap_id + '.png'
    if filename not in os.listdir('images'):
        print(filename)

In [64]:
df_filtered['img_url'] = df_filtered['gmap_id'] + '.png'

<ipython-input-64-d545b7ebf3e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['img_url'] = df_filtered['gmap_id'] + '.png'


In [65]:
df_filtered

,user_id,gmap_id,rating,text,img_url
0,108979378435066002945,0x4cb549e8877cf0d7:0xe8f003e6d73392ae,5,Quintessential Vermont experience (even for Ve...,0x4cb549e8877cf0d7:0xe8f003e6d73392ae.png
1,101856865551768948430,0x89e024446398691f:0x4011d3a0f8636036,1,I had them replace two broken studs on my righ...,0x89e024446398691f:0x4011d3a0f8636036.png
2,107787438275893909028,0x4cb446270577ed85:0x8b3dfc93cc4846a8,5,Kingdom Recovery Center is a place for people ...,0x4cb446270577ed85:0x8b3dfc93cc4846a8.png
3,117544587328467760561,0x4cb4462e2f53cf2f:0xfbe0971a8791662,5,"The river is cool, no, the river is cold. It i...",0x4cb4462e2f53cf2f:0xfbe0971a8791662.png
4,105058591299841405589,0x4cb42a9f5df6385b:0x534ca42c250f0a05,5,Took the Mad Brook trail. Definitely not the e...,0x4cb42a9f5df6385b:0x534ca42c250f0a05.png
...,...,...,...,...,...
15448,105910321509155624260,0x89e1c79c60aef325:0xbed8bfc41018f020,5,"I love coming here, no matter what we pick fro...",0x89e1c79c60aef325:0xbed8bfc41018f020.png
15449,113393116818166217694,0x89e1c79c60aef325:0xbed8bfc41018f020,4,"We loved the petting zoo, the kids always have...",0x89e1c79c60aef325:0xbed8bfc41018f020.png
15450,103909822198684259976,0x89e1c79c60aef325:0xbed8bfc41018f020,5,"Friendly staff, tractor rides, 50¢ petting zoo...",0x89e1c79c60aef325:0xbed8bfc41018f020.png
15451,114401403353668611090,0x89e1c79c60aef325:0xbed8bfc41018f020,4,We were here for one of there special customer...,0x89e1c79c60aef325:0xbed8bfc41018f020.png


In [66]:
df_filtered.to_csv('reviews_vermont.csv', index=False)